In [1]:
import os

import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "gestion_immo.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [ ]:
from agence.models import Acheteur, Agent, Utilisateur, Vendeur

utilisateur = Utilisateur.objects.create(
    nom="Bambelle", prenom="Larry", telephone="+33158350027", email="bambelle.larry@youhou.fr"
)
utilisateur

In [14]:
Utilisateur.objects.all()

<QuerySet [<Utilisateur: Larry Bambelle  (bambelle.larry@youhou.fr, +33 1 58 35 00 27)>]>

In [5]:
from agence.models import Adresse
from django.db.models import Count

Adresse.objects.count()

22544743

In [15]:
acheteur = Acheteur.objects.create(utilisateur=utilisateur)

IntegrityError: UNIQUE constraint failed: agence_acheteur.utilisateur_id

In [4]:
from django.conf import settings


def create_url():
    db_config = settings.DATABASES["default"]
    db_engine = db_config["ENGINE"]
    db_name = db_config["NAME"]
    db_user = db_config["USER"]
    db_password = db_config["PASSWORD"]
    db_host = db_config["HOST"]
    db_port = db_config["PORT"]
    if db_engine == "django.db.backends.postgresql":
        return f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
    elif db_engine == "django.db.backends.mysql":
        return f"mysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
    elif db_engine == "django.db.backends.sqlite3":
        return f"sqlite:///{db_name}"
    else:
        raise ValueError("Unsupported database engine")


create_url()

'sqlite:////home/arnaud-ma/cours/M1/S2/projetBDD2025/db.sqlite3'

In [26]:
import pandas as pd


def get_df():
    return pd.read_csv(
        "adresses-france.csv",
        chunksize=1_000_000,
        sep=";",
        usecols=[
            "id_fantoir",
            "rep",
            "nom_voie",
            "code_postal",
            "code_insee",
            "nom_commune",
            "lon",
            "lat",
            "alias",
            "numero",
        ],
        dtype={
            "id": "string",
            "id_fantoir": "string",
            "numero": "string",
            "rep": "string",
            "nom_voie": "string",
            "code_postal": "string",
            "code_insee": "string",
            "code_commune": "string",
            "nom_commune": "string",
            "lon": "float64",
            "lat": "float64",
            "alias": "string",
        },
    )

In [33]:
import numpy as np

X = pd.DataFrame()

for chunk in get_df():
    chunk: pd.DataFrame
    x = chunk[chunk["id_fantoir"].notna()]
    # x = chunk[["code_insee", "nom_commune", "code_postal"]].fillna("")
    X = pd.concat([X, x], ignore_index=True).drop_duplicates(subset=["code_insee"])
    break

In [34]:
X

,id_fantoir,numero,rep,nom_voie,code_postal,code_insee,nom_commune,lon,lat,alias
0,01001_0165,19,<NA>,Route de la Fontaine,01400,01001,L'Abergement-Clémenciat,4.920296,46.151585,<NA>
428,01005_A067,1,<NA>,Lotissement le Parc des Dombes,01330,01005,Ambérieux-en-Dombes,4.901308,45.994579,<NA>
1044,01002_0050,38,<NA>,Boucle du Closet,01640,01002,L'Abergement-de-Varey,5.427759,46.004630,<NA>
1265,01004_0162,1,<NA>,Allée du Beau Fayard,01500,01004,Ambérieu-en-Bugey,5.363262,45.961432,<NA>
5979,01330_0120,10,<NA>,Rue de la Buissiere,01260,01330,Ruffieu,5.660834,45.998508,<NA>
...,...,...,...,...,...,...,...,...,...,...
860316,06092_0052,1,<NA>,Carriera Dau Gourguet,06440,06092,Peillon,7.382215,43.777943,<NA>
860834,06089_0206,1,<NA>,Chemin des Colles,06650,06089,Opio,6.990306,43.656690,<NA>
861787,06091_0115,1,<NA>,Rue des Moulins,06440,06091,Peille,7.401760,43.802673,<NA>
863239,06090_0352,1,<NA>,Allée des Géraniums,06580,06090,Pégomas,6.930533,43.599409,<NA>


In [14]:
from sqlalchemy import create_engine

engine = create_engine(create_url())

X = X.rename(
    columns={
        "code_insee": "code_insee",
        "nom_commune": "nom",
        "code_postal": "code_postal",
    }
)


with engine.connect() as conn:
    X.to_sql(
        "agence_commune",
        con=conn,
        if_exists="append",
        index=False,
    )

with create_engine(create_url()).connect() as conn:
    communes_df = pd.read_sql_table("agence_commune", conn, columns=["id", "code_insee"])
communes_df

,id,code_insee
0,1,01001
1,3,01002
2,4,01004
3,2,01005
4,391,01006
...,...,...
34910,34914,98810
34911,34913,98811
34912,34911,98817
34913,34912,98818


In [12]:
X

,code_insee,nom,code_postal
0,01001,L'Abergement-Clémenciat,01400
1,01005,Ambérieux-en-Dombes,01330
2,01002,L'Abergement-de-Varey,01640
3,01004,Ambérieu-en-Bugey,01500
4,01330,Ruffieu,01260
...,...,...,...
48661,98817,Le Mont-Dore,98809
56280,98818,Nouméa,98800
79093,98811,Koné,98859
81172,98810,Kaala-Gomen,98817


In [ ]:
import numpy as np

X2 = pd.DataFrame()
for i, chunk in enumerate(get_df()):
    print(f"Chunk {i}")
    chunk: pd.DataFrame
    # Voies

    df_voies = (
        chunk[["id_fantoir", "nom_voie", "code_insee"]]
        .dropna(subset=["id_fantoir"])
        .drop_duplicates(subset=["id_fantoir"])
        .merge(communes_df, on="code_insee", how="left")
        .rename(columns={"id": "commune_id", "nom_voie": "nom"})[
            ["id_fantoir", "nom", "commune_id"]
        ]
    )
    X2 = pd.concat([X2, df_voies], ignore_index=True).drop_duplicates(subset=["id_fantoir"])
    # df_voies.to_sql(
    #     "agence_voie",
    #     con=engine,
    #     if_exists="append",
    #     index=False,
    #     method="multi",
    #     chunksize=10_000,
    # )
    print(f"{len(X2):,}")


Chunk 0
71,942
Chunk 1
140,404
Chunk 2
199,545
Chunk 3
275,539
Chunk 4
348,717
Chunk 5
422,972
Chunk 6
480,939
Chunk 7
536,572
Chunk 8
599,515
Chunk 9
671,920
Chunk 10
749,763
Chunk 11
809,667
Chunk 12
872,092
Chunk 13
942,909
Chunk 14
992,683
Chunk 15
1,040,178
Chunk 16
1,090,017
Chunk 17
1,162,785
Chunk 18
1,232,768
Chunk 19
1,301,153
Chunk 20
1,357,638
Chunk 21
1,423,844
Chunk 22
1,482,199
Chunk 23
1,553,071
Chunk 24
1,598,736
Chunk 25
1,638,431
Chunk 26
1,638,793


In [47]:
X2.to_sql(
    "agence_voie",
    con=engine,
    if_exists="append",
    index=False,
    method="multi",
    chunksize=10_000,
)

1638793

In [5]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine(create_url())
with engine.connect() as conn:
    adresses_df = pd.read_sql_table("agence_voie", conn)

In [ ]:
adresses_df.dtypes

id             int64
id_fantoir    object
nom           object
commune_id     int64
dtype: object

In [8]:
adresses_df.astype(str).agg("".join, axis=1).str.lower().str.replace(" ", "")

0               101001_0165routedelafontaine1
1               201001_0115routedeclémenciat1
2                301001_0370impassedessoyeux1
3                401001_0250impassedesmerles1
4                501001_0175routedesgaraudes1
                          ...                
1638788     163878997127_b094sandyground34906
1638789     163879097127_b099mornevalois34906
1638790    163879197127_b104zadebellevue34906
1638791    163879297127_b107howellcenter34906
1638792    163879397127_b111zahopeestate34906
Length: 1638793, dtype: object

In [22]:
import requests


params = {"text": "14 rue des pervenches Monts"}
response = requests.get("https://data.geopf.fr/geocodage/completion/", params, timeout=5)
response.raise_for_status()

https://data.geopf.fr/geocodage/completion/?text=a&type=StreetAddress&maximumResponses=10
https://data.geopf.fr/geocodage/completion/?text=a&type=StreetAddress&maximumResponses=10

In [23]:
result = response.json()
result.get("status")

'OK'

In [24]:
result

{'status': 'OK',
 'results': [{'x': 0.656587,
   'y': 47.273759,
   'country': 'StreetAddress',
   'city': 'Monts',
   'oldcity': '',
   'kind': 'housenumber',
   'zipcode': '37260',
   'street': 'Rue des Pervenches',
   'metropole': True,
   'fulltext': '14 Rue des Pervenches, 37260 Monts',
   'classification': 7},
  {'x': 3.884303,
   'y': 43.59938,
   'country': 'StreetAddress',
   'city': 'Montpellier',
   'oldcity': '',
   'kind': 'housenumber',
   'zipcode': '34000',
   'street': 'Rue des Pervenches',
   'metropole': True,
   'fulltext': '14 Rue des Pervenches, 34000 Montpellier',
   'classification': 7},
  {'x': 166.571984,
   'y': -22.270896,
   'country': 'StreetAddress',
   'city': 'Le Mont-Dore',
   'oldcity': '',
   'kind': 'housenumber',
   'zipcode': '98809',
   'street': 'Rue des Pervenches',
   'metropole': False,
   'fulltext': '14 Rue des Pervenches, 98809 Le Mont-Dore',
   'classification': 7},
  {'x': 2.748622,
   'y': 48.002376,
   'country': 'StreetAddress',
   'c

In [25]:
results = result.get("results")
[value for result in results if (value := result.get("fulltext"))]

['14 Rue des Pervenches, 37260 Monts',
 '14 Rue des Pervenches, 34000 Montpellier',
 '14 Rue des Pervenches, 98809 Le Mont-Dore',
 'Rue des Pervenches, 45200 Montargis',
 'Rue des Pervenches, 42600 Montbrison',
 'Rue des Pervenches, 97354 Remire-Montjoly',
 'Rue des Pervenches, 01090 Montmerle-sur-Saône',
 'Rue des Pervenches, 54350 Mont-Saint-Martin',
 'Montée des Pervenches, 97133 Saint-Barthélemy',
 '14 Avenue des Pervenches, 93370 Montfermeil']